In [115]:
import numpy as np

pose2d = np.load("./data/fs_tas/features/2dvpd/men_olympic_short_program_2010_01_00011475_00015700.npy")
pose3d = np.load("./data/fs_tas/features/pose3d/men_olympic_short_program_2010_01_00011475_00015700.npy")

In [127]:
print(pose2d.shape, pose3d.shape)

(4225, 52) (4225, 54)


In [147]:
data_reshaped = pose3d.reshape(-1, 18, 3)[1000:1100,:,:] # local, euler angles # 2000, 22000

In [148]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def plot_3d(pose_3d):
    all_poses = pose_3d
    poses = []

    for frame in range(len(all_poses)):
        vals = all_poses[frame]
        joint_positions = vals[:17].copy()
        euler_angles = vals[17].copy()
        
        rot = R.from_euler('xyz', euler_angles, degrees=False)
        # rot = R.from_rotvec(-euler_angles, degrees=True)
        joint_positions = rot.apply(joint_positions)
        
        poses.append(joint_positions)

    return np.array(poses)

In [149]:
poses_global = plot_3d(data_reshaped)
poses_global[0]

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.1006977 ,  0.06082883,  0.04249604],
       [ 0.41956523,  0.06504617, -0.40193571],
       [ 0.14529508,  0.67268797, -0.62872569],
       [-0.09768949, -0.07147211, -0.03843642],
       [ 0.34618816,  0.00128235, -0.4528931 ],
       [ 0.20232638,  0.19693299, -0.84600965],
       [ 0.1278628 , -0.32472052,  0.23286443],
       [ 0.26876279, -0.66081341,  0.47285089],
       [ 0.54390968, -0.97989433,  0.50566116],
       [ 0.5290155 , -1.02669336,  0.56144277],
       [ 0.09653294, -0.70043098,  0.4325931 ],
       [-0.01965399, -0.54567086,  0.09682585],
       [ 0.15675821, -0.54497702, -0.17256443],
       [ 0.40595366, -0.58604545,  0.47034126],
       [ 0.60172839, -0.49534365,  0.0546133 ],
       [ 0.75353626, -0.59662836, -0.27882309]])

In [175]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def get_canonical_orientation(X, torso_forward_vec, spine_up_vec,
                              interp_start=45, interp_range=30):
    X_zm = X - np.mean(X, axis=0).flatten()
    _, _, V = np.linalg.svd(X_zm)
    torso_forward_vec = -V[2, :] if V[2, :].dot(torso_forward_vec) < 0 else V[2, :]
    spine_up_vec = -V[0, :] if V[0, :].dot(spine_up_vec) < 0 else V[0, :]
    torso_pitch = np.degrees(np.arcsin(torso_forward_vec[2]))
    if torso_pitch > interp_start:
        if torso_pitch < interp_start + interp_range:
            theta = (torso_pitch - interp_start) / interp_range
            return theta * -spine_up_vec + (1. - theta) * torso_forward_vec
        else:
            return -spine_up_vec
    elif torso_pitch < -interp_start:
        if torso_pitch > -interp_start - interp_range:
            theta = (-torso_pitch - interp_start) / interp_range
            return theta * spine_up_vec + (1. - theta) * torso_forward_vec
        else:
            return spine_up_vec
    else:
        return torso_forward_vec

def h36m_global_to_local(pose_3d_global):
    """
    Convert H36M global joint positions to local with canonical orientation alignment.
    
    H36M 17-joint skeleton:
    0: Hip, 1: RHip, 2: RKnee, 3: RAnkle, 4: LHip, 5: LKnee, 6: LAnkle  
    7: Spine, 8: Thorax, 9: Neck, 10: Head, 11: LShoulder, 12: LElbow, 13: LWrist
    14: RShoulder, 15: RElbow, 16: RWrist
    
    Args:
        pose_3d_global: (nframe, 17, 3) array of global joint positions
    
    Returns:
        pose_3d_local: (nframe, 18, 3) array where:
                      - First row (index 0) contains euler angles (rx, ry, rz)
                      - Remaining 17 rows contain local joint positions
    """
    nframes, njoints, _ = pose_3d_global.shape
    pose_3d_local = np.zeros((nframes, 18, 3))
    
    Z_UNIT = np.array([0, 0, 1])
    HIP, SPINE, NECK, L_HIP, R_HIP, L_SHOULDER, R_SHOULDER = 0, 7, 9, 4, 1, 11, 14
    
    for frame in range(nframes):
        xyz = pose_3d_global[frame].copy()
        
        root_position = xyz[HIP].copy()
        xyz = xyz - root_position
        
        key_joints = xyz[[HIP, SPINE, NECK, L_SHOULDER, R_SHOULDER, L_HIP, R_HIP], :]
        torso_forward_vec = np.cross(xyz[L_SHOULDER, :] - xyz[HIP, :], xyz[R_SHOULDER, :] - xyz[HIP, :])
        spine_up_vec = xyz[NECK, :] - xyz[HIP, :]
        
        forward_vec = get_canonical_orientation(key_joints, torso_forward_vec, spine_up_vec)
        forward_vec[2] = 0
        forward_vec = forward_vec / np.linalg.norm(forward_vec)
        lateral_vec = np.cross(Z_UNIT, forward_vec)
        forward_vec = np.cross(lateral_vec, Z_UNIT)

        lateral_vec /= np.linalg.norm(lateral_vec)
        forward_vec /= np.linalg.norm(forward_vec)

        rot_mat = np.stack([lateral_vec, forward_vec, Z_UNIT], axis=1)

        if np.linalg.det(rot_mat) < 0:
            rot_mat[:, 2] *= -1  # Flip Z to restore right-handedness
        
        euler_angles = R.from_matrix(rot_mat).as_euler('xyz', degrees=False)
        
        pose_3d_local[frame, 17, :] = euler_angles
        pose_3d_local[frame, :17, :] = xyz
    
    return pose_3d_local

In [176]:
poses_local = h36m_global_to_local(pose_3d_global=poses_global)
print(poses_local[1][17], data_reshaped[1][17])

[-3.14159265  0.         -2.92402609] [-0.40160613 -0.27975454  0.35582679]


In [215]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# Joint indices for H36M 17-joint format
HIP, NECK, L_HIP, R_HIP, L_SHOULDER, R_SHOULDER = 0, 9, 4, 1, 11, 14


def estimate_alignment_euler(poses, order="xyz"):
    """
    Estimate canonical alignment rotation from Human3.6M pose.

    Parameters:
    -----------
    poses : ndarray (17, 3) — single frame of joint positions
    order : str — Euler rotation order ('xyz', 'zyx', etc.)

    Returns:
    --------
    euler_angles : ndarray (3,) — Euler angles in radians
    """
    # 1. Extract key joints
    r_hip = poses[R_HIP]
    l_hip = poses[L_HIP]
    neck = poses[NECK]
    hip = poses[HIP]

    # 2. Estimate local torso axes
    x_axis = r_hip - l_hip  # right-to-left (→ x)

    z_axis = neck - hip  # up (→ z)

    # 3. Compute y as perpendicular to x and z
    y_axis = np.cross(z_axis, x_axis) # forward

    # 4. Orthonormalize
    x_axis = x_axis / np.linalg.norm(x_axis)
    y_axis = y_axis / np.linalg.norm(y_axis)
    z_axis = z_axis / np.linalg.norm(z_axis)
    
    # Re-orthogonalize (in case of drift)
    y_axis = np.cross(z_axis, x_axis)  # ensures right-handed system

    # 5. Form rotation matrix (columns = basis vectors)
    R_align = np.stack([x_axis, y_axis, z_axis], axis=1)  # shape (3, 3)

    # 6. Fix if left-handed (negative determinant)
    if np.linalg.det(R_align) < 0:
        R_align[:, 0] *= -1  # Flip x to fix handedness

    # 7. Convert to Euler angles
    return R.from_matrix(R_align).as_euler(order, degrees=False)

In [200]:
HIP, NECK, L_HIP, R_HIP, L_SHOULDER, R_SHOULDER = 0, 9, 4, 1, 11, 14
euler_angle = estimate_alignment_euler(
    poses_global[0]
)

print(euler_angle, data_reshaped[0][17])

NameError: name 'zyx' is not defined

In [104]:
rot = R.from_euler('xyz', euler_angle, degrees=False).inv()
poses_local = rot.apply(poses_global[0])